[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pinecone-io/examples/blob/master/integrations/groq/groq-llama-3-rag.ipynb) [![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/nbviewer-shield.svg)](https://nbviewer.org/github/pinecone-io/examples/blob/master/integrations/groq/groq-llama-3-rag.ipynb)

# RAG with Groq and Llama 3

To begin, we setup our prerequisite libraries.

In [2]:
!pip install -qU \
    groq==0.8.0 \
    "semantic-router[local]==0.0.45" \
    pinecone-client==4.1.0

In [14]:
!pip install datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.9 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


## Data Preparation

We start by downloading a dataset that we will encode and store. The dataset [`jamescalam/ai-arxiv2-semantic-chunks`](https://huggingface.co/datasets/jamescalam/ai-arxiv2-semantic-chunks) contains scraped data from many popular ArXiv papers centred around LLMs and GenAI.

In [3]:
file_path = './data/job_title_des.csv'

In [3]:
import pandas as pd

df = pd.read_csv(file_path)

# Check for null values in each column
null_counts = df.isnull().sum()
print(null_counts)

Unnamed: 0         0
Job Title          0
Job Description    0
dtype: int64


In [5]:
df.head()

,Unnamed: 0,Job Title,Job Description
0,0,Flutter Developer,We are looking for hire experts flutter develo...
1,1,Django Developer,PYTHON/DJANGO (Developer/Lead) - Job Code(PDJ ...
2,2,Machine Learning,"Data Scientist (Contractor)\n\nBangalore, IN\n..."
3,3,iOS Developer,JOB DESCRIPTION:\n\nStrong framework outside o...
4,4,Full Stack Developer,job responsibility full stack engineer – react...


In [10]:
df = df.rename(columns={df.columns[0]: 'id'})
df.head()

,id,Job Title,Job Description
0,0,Flutter Developer,We are looking for hire experts flutter develo...
1,1,Django Developer,PYTHON/DJANGO (Developer/Lead) - Job Code(PDJ ...
2,2,Machine Learning,"Data Scientist (Contractor)\n\nBangalore, IN\n..."
3,3,iOS Developer,JOB DESCRIPTION:\n\nStrong framework outside o...
4,4,Full Stack Developer,job responsibility full stack engineer – react...


In [11]:
data_list = df.to_dict('records')
data_list[0]

{'id': 0,
 'Job Title': 'Flutter Developer',
 'Job Description': 'We are looking for hire experts flutter developer. So you are eligible this post then apply your resume.\nJob Types: Full-time, Part-time\nSalary: ₹20,000.00 - ₹40,000.00 per month\nBenefits:\nFlexible schedule\nFood allowance\nSchedule:\nDay shift\nSupplemental Pay:\nJoining bonus\nOvertime pay\nExperience:\ntotal work: 1 year (Preferred)\nHousing rent subsidy:\nYes\nIndustry:\nSoftware Development\nWork Remotely:\nTemporarily due to COVID-19'}

In [44]:
from datasets import load_dataset

dataset = load_dataset("csv", data_files={"train": file_path})

In [45]:
data = dataset['train']
print(type(data))

<class 'datasets.arrow_dataset.Dataset'>


In [43]:
data[0]

{'id': 0,
 'metadata': {'Job Description': 'We are looking for hire experts flutter developer. So you are eligible this post then apply your resume.\nJob Types: Full-time, Part-time\nSalary: ₹20,000.00 - ₹40,000.00 per month\nBenefits:\nFlexible schedule\nFood allowance\nSchedule:\nDay shift\nSupplemental Pay:\nJoining bonus\nOvertime pay\nExperience:\ntotal work: 1 year (Preferred)\nHousing rent subsidy:\nYes\nIndustry:\nSoftware Development\nWork Remotely:\nTemporarily due to COVID-19',
  'Job Title': 'Flutter Developer'}}

Format the data into the format we need, this will contain `id`, `text` (which we will embed), and `metadata`.

In [46]:
data = data.map(lambda x, i: {
    "id": i,  # Use the index as the ID
    "metadata": {
         "Job Description": x["Job Description"],
        "Job Title": x["Job Title"],
    }
}, with_indices=True)


# drop uneeded columns
data = data.remove_columns([
    "Job Title", "Job Description" , "Unnamed: 0",
 ])
data

Map:   0%|          | 0/2277 [00:00<?, ? examples/s]

Dataset({
    features: ['id', 'metadata'],
    num_rows: 2277
})

In [47]:
data[1]

{'id': 1,
 'metadata': {'Job Description': 'PYTHON/DJANGO (Developer/Lead) - Job Code(PDJ - 04)\nStrong Python experience in API development (REST/RPC).\nExperience working with API Frameworks (Django/flask).\nExperience evaluating and improving the efficiency of programs in a Linux environment.\nAbility to effectively handle multiple tasks with a high level of accuracy and attention to detail.\nGood verbal and written communication skills.\nWorking knowledge of SQL.\nJSON experience preferred.\nGood knowledge in automated unit testing using PyUnit.',
  'Job Title': 'Django Developer'}}

We need to define an embedding model to create our embedding vectors for retrieval, for that we will be using a variation of the `e5-base` model with a longer context length of `4k` tokens. Ideally we should be running this on GPU for optimal runtimes.

In [25]:
from semantic_router.encoders import HuggingFaceEncoder

encoder = HuggingFaceEncoder(name="dwzhu/e5-base-4k")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/228 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/691 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/225M [00:00<?, ?B/s]

We can check whether our `encoder` will use `cpu` or a `cuda` GPU (where available).

In [26]:
encoder.device

'cuda'

We can create embeddings now like so:

In [48]:
embeds = encoder(["this is a test etermine the dimensionality of your embeddings test test stet etermine the dimensionality of your embeddings"])

We can view the dimensionality of our returned embeddings, which we'll need soon when initializing our vector index:

In [49]:
dims = len(embeds[0])
dims

768

Now we create our vector DB to store our vectors. For this we need to get a [free Pinecone API key](https://app.pinecone.io) — the API key can be found in the "API Keys" button found in the left navbar of the Pinecone dashboard.

In [29]:
import os
import getpass
from pinecone import Pinecone

# initialize connection to pinecone (get API key at app.pinecone.io)
api_key = os.getenv("PINECONE_API_KEY") or getpass.getpass("Enter your Pinecone API key: ")

# configure client
pc = Pinecone(api_key=api_key)

Enter your Pinecone API key: ··········


Now we setup our index specification, this allows us to define the cloud provider and region where we want to deploy our index. You can find a list of all [available providers and regions here](https://docs.pinecone.io/docs/projects).

In [30]:
from pinecone import ServerlessSpec

spec = ServerlessSpec(
    cloud="aws", region="us-east-1"
)

Creating an index, we set `dimension` equal to the dimensionality of our encoder (`384`), and use a `metric` also compatible with the model (this can be `cosine`). We also pass our `spec` to index initialization.

In [50]:
import time

index_name = "groq-llama-3-rag"
existing_indexes = [
    index_info["name"] for index_info in pc.list_indexes()
]

# check if index already exists (it shouldn't if this is first time)
if index_name not in existing_indexes:
    # if does not exist, create index
    pc.create_index(
        index_name,
        dimension=dims,
        metric='cosine',
        spec=spec
    )
    # wait for index to be initialized
    while not pc.describe_index(index_name).status['ready']:
        time.sleep(1)

# connect to index
index = pc.Index(index_name)
time.sleep(1)
# view index stats
index.describe_index_stats()

{'dimension': 768,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

We can see the index is currently empty with a `total_vector_count` of `0`. We can begin populating it with our embeddings.

In [51]:
from tqdm.auto import tqdm

batch_size = 128  # how many embeddings we create and insert at once

for i in tqdm(range(0, len(data), batch_size)):
    # find end of batch
    i_end = min(len(data), i+batch_size)
    # create batch
    batch = data[i:i_end]
    # create embeddings
    chunks = [f'{x["Job Title"]}: {x["Job Description"]}' for x in batch["metadata"]]
    embeds = encoder(chunks)
    assert len(embeds) == (i_end-i)
    to_upsert = list(zip(map(str, batch["id"]), embeds, batch["metadata"]))
    # upsert to Pinecone
    index.upsert(vectors=to_upsert)

  0%|          | 0/18 [00:00<?, ?it/s]

Now let's test retrieval!

In [52]:
def get_docs(query: str, top_k: int) -> list[str]:
    # encode query
    xq = encoder([query])
    # search pinecone index
    res = index.query(vector=xq, top_k=top_k, include_metadata=True)
    # get doc text
    docs = [x["metadata"]['Job Description'] for x in res["matches"]]
    return docs

Modify top_k to see more outputs

In [54]:
query = "can you give me job description of Java Developer?"
docs = get_docs(query, top_k=1)
print("\n---\n".join(docs))

Responsibilities for this profile are shared as below:
Work experience as a senior Java developer or similar role.
Having good knowledge of any RDBMS (e.g. Oracle, MySQL) * Experience Designing, building and testing Java EE application.
Knowledge of Java frameworks like JSF and Spring MVC.
Experience with Object Orient Design (OOD).
Ability to work independently
Good delegations and time management skills.
Has a good attitude with keens to adapt to new technologies.
Job Types: Full-time, Fresher, Walk-In
Salary: ₹8,000.00 - ₹10,000.00 per month
Schedule:
Day shift
Experience:
total work: 1 year (Preferred)
JSP: 1 year (Preferred)
java: 1 year (Preferred)
Education:
Bachelor's (Preferred)
Licence:
Java SE 7 Programmer (Preferred)
Work Remotely:
No


Our retrieval component works, now let's try feeding this into a Llama 3 70B model hosted by Groq to produce an answer.

In [1]:
from groq import Groq

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY") or getpass.getpass("Enter your Groq API key: ")

groq_client = Groq(api_key=os.environ["GROQ_API_KEY"])

ModuleNotFoundError: No module named 'groq'

Now we can generate responses using Llama 3, we'll wrap this logic into a help function called `generate`:

In [ ]:
def generate(query: str, docs: list[str]):
    system_message = (
        "You are a helpful assistant that generates job descriptions "
        "context provided below.\n\n"
        "CONTEXT:\n"
        "\n---\n".join(docs)
    )
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": query}
    ]
    # generate response
    chat_response = groq_client.chat.completions.create(
        model="gemma2-9b-it",
        messages=messages
    )
    return chat_response.choices[0].message.content

In [ ]:
query = "Create a job description for a web developer in the tech industry with friendly tone"
docs = get_docs(query, top_k=5)
out = generate(query=query, docs=docs)
print(out)

Score: 0

Response: This response missed the core of the task. 

While the provided context is about a unique chatbot format, the actual request is to generate a job description. 

Here's how to approach this task: 

1. **Understand the Role:**  A web developer builds and maintains websites. This can involve front-end development (what users see), back-end development (the behind-the-scenes functionality), or both.

2. **Identify Key Skills:** 
    *  **Programming Languages:** HTML, CSS, JavaScript, Python (or other relevant languages)
    *  **Frameworks/Libraries:** React, Angular, Node.js, Django, etc.
    *  **Databases:** MySQL, MongoDB, PostgreSQL 
    *  **Version Control:** Git

3. **Craft a Friendly Description:**

  Here's an example:

  
  **Web Developer - Join Our Team!**
  
  We're looking for a passionate and creative Web Developer to join our growing team!  You'll be building and improving our websites, making sure they're user-friendly, engaging, and constantly up-to-

Don't forget to delete your index when you're done to save resources!

In [32]:
pc.delete_index(index_name)

---